In [14]:
import json
import datetime
import os
import pandas as pd
from collections import Counter

In [48]:
datfiles=os.listdir('data/')

In [63]:
def convert_timestring(x):
    x=x[:-6]+''.join(x[-6:].split(':'))
    return datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z')
    
def convert_timestring_utc(x):
    if x is None:
        return x
    x=x[:-6]+''.join(x[-6:].split(':'))
    return datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z').astimezone(tz=datetime.timezone.utc)

In [66]:
order_dict={}
line_item_dict={}
user_dict={}

for fn in datfiles:
    with open('data/'+fn,'r') as f:
        print(fn)
        data=json.load(f)
        orders=data['orders']
        for order in orders:
            order_id=order['id']
            line_item_list=order['line_items']
            line_item_total=len(line_item_list)
            item_total=sum([x['quantity'] for x in line_item_list])
            processed_at=convert_timestring_utc(order['processed_at'])
            cancelled_at=convert_timestring_utc(order['cancelled_at'])
            if order_id in order_dict:
                print('Error: Duplicative order_id: '+order_id)
                continue
            else:
                order_dict[order_id]={k:v for k,v in order.items() if k in set(['line_items'])}
                order_dict[order_id]['line_item_total']=line_item_total
                order_dict[order_id]['processed_at']=processed_at
                order_dict[order_id]['cancelled_at']=cancelled_at
            
            for line_item in line_item_list:
                line_item_id=line_item['id']
                if line_item_id in line_item_dict:
                    print('Error: Duplicative line_item_id: '
                          +line_item_id+', order_id: '+order_id)
                    continue
                else:
                    line_item_dict[line_item_id]={k:v for k,v in line_item.items() if k!='id'}
                    line_item_dict[line_item_id]['order_id']=order_id
                    
            user_id=order['user_id']
            min_processed_at=processed_at
            max_processed_at=processed_at
            if user_id not in user_dict:
                user_dict[user_id]={k: v for k, v in order.items() if k in set(['customer_locale','buyer_accepts_marketing','contact_email','phone'])}
                user_dict[user_id]['total_orders']=1
                user_dict[user_id]['total_line_items']=line_item_total
                user_dict[user_id]['total_items']=item_total
                
            else:
                user_dict[user_id]['total_orders']+=1
                user_dict[user_id]['total_line_items']+=line_item_total
                user_dict[user_id]['total_items']+=item_total
                prev_min=user_dict[user_id]['min_processed_at']
                prev_max=user_dict[user_id]['max_processed_at']
                min_processed_at=min(prev_min,min_processed_at)
                max_processed_at=max(prev_max,max_processed_at)
            user_dict[user_id]['min_processed_at']=min_processed_at
            user_dict[user_id]['max_processed_at']=max_processed_at
            user_dict[user_id]['total_days']=(max_processed_at-min_processed_at).days+1

2017-10-30.json
2017-11-01.json
2017-11-02.json
2017-11-03.json
2017-11-04.json
2017-11-05.json
2017-11-06.json
2017-11-07.json
2017-11-08.json
2017-11-09.json
2017-11-10.json
2017-11-11.json
2017-11-12.json
2017-11-13.json
2017-11-14.json
2017-11-15.json
2017-11-16.json
2017-11-17.json
2017-11-18.json
2017-11-19.json
2017-11-20.json
2017-11-21.json
2017-11-22.json
2017-11-24.json
2017-11-25.json
2017-11-26.json
2017-11-27.json
2017-11-28.json
2017-11-29.json
2017-11-30.json
2017-12-30.json


In [67]:

order_df=pd.DataFrame.from_dict(order_dict,orient='index')
Counter(order_df['cancelled_at'])  

Counter({None: 13438})

In [45]:

line_item_df=pd.DataFrame.from_dict(line_item_dict,orient='index')

In [ ]:
How and when is each record coming in?

variables to reformat:
    order and order_number are always 1000 apart
    closed_at
    created_at
    updated_at
    processed_at
    cancelled_at

In [ ]:
ignore variables that are mostly null

In [37]:
{k: v for k, v in line_item.items() if k!='id'}

{'product_id': 9086871491, 'quantity': 1, 'variant_id': 31894084547}

In [42]:
{k: v for k, v in line_item.items() if k not in set(['id','variant_id'])}

{'product_id': 9086871491, 'quantity': 1}

In [43]:
{k: v for k, v in line_item.items() if k in set(['id','variant_id'])}

{'id': 23337631747, 'variant_id': 31894084547}

In [46]:
{k: v for k, v in order.items() if k in set(['id','variant_id'])}

{'id': 11674779651}

In [7]:
order.items()

dict_items([('id', 11748933635), ('email', 'censored@censored.com'), ('closed_at', None), ('created_at', '2017-10-30T19:58:29-04:00'), ('updated_at', '2017-10-30T19:59:30-04:00'), ('number', 97171), ('note', None), ('token', 'censored'), ('gateway', 'cash'), ('test', False), ('total_price', '130.64999999999998'), ('subtotal_price', '120.0'), ('total_weight', 6), ('total_tax', '10.649999999999999'), ('taxes_included', False), ('currency', 'USD'), ('financial_status', 'paid'), ('confirmed', True), ('total_discounts', '0.00'), ('total_line_items_price', '120.0'), ('cart_token', 'censored'), ('buyer_accepts_marketing', False), ('name', 'GLO37-2010000000'), ('referring_site', None), ('landing_site', None), ('cancelled_at', None), ('cancel_reason', None), ('total_price_usd', '130.64999999999998'), ('checkout_token', 'censored'), ('reference', '9925E904-4A8D-4520-9F2F-46B847BDE46D'), ('user_id', 110585347), ('location_id', 371291), ('source_identifier', '371291-37-20100'), ('source_url', None

In [ ]:
'customer_locale','buyer_accepts_marketing','contact_email','phone'
order_history+=1 


order_number

order costs

TOTAL SPEND

media vars

order status? paid or not... fulfillment status

In [50]:

user_df=pd.DataFrame.from_dict(user_dict,orient='index')
user_df.head(100)

,buyer_accepts_marketing,phone,customer_locale,contact_email,total_orders,total_line_items,total_items,min_processed_at,max_processed_at,total_days
8055575,False,555-555-5555,None,censored@censored.com,243,591,642,2017-11-02 19:00:56+00:00,2017-11-22 00:55:08+00:00,20
31994179,True,555-555-5555,None,censored@censored.com,5458,13770,14907,2017-11-01 17:44:02+00:00,2017-12-30 22:30:26+00:00,60
110420419,True,555-555-5555,None,censored@censored.com,24,63,67,2017-11-22 18:22:41+00:00,2017-11-29 17:46:21+00:00,7
110420483,False,555-555-5555,None,censored@censored.com,296,743,810,2017-10-30 23:00:28+00:00,2017-11-30 21:47:13+00:00,31
110420675,False,555-555-5555,None,censored@censored.com,184,438,462,2017-11-01 16:12:04+00:00,2017-11-16 00:53:49+00:00,15
110420803,True,555-555-5555,None,censored@censored.com,85,214,230,2017-11-04 15:09:30+00:00,2017-11-12 17:01:58+00:00,9
110420867,True,555-555-5555,None,censored@censored.com,113,272,283,2017-10-30 22:47:09+00:00,2017-11-28 01:06:09+00:00,29
110420995,False,555-555-5555,None,censored@censored.com,413,989,1087,2017-11-03 21:59:59+00:00,2017-12-30 22:14:50+00:00,58
110421059,True,555-555-5555,None,censored@censored.com,650,1536,1660,2017-11-01 20:00:18+00:00,2017-11-29 23:48:58+00:00,29
110488387,False,555-555-5555,None,censored@censored.com,212,532,606,2017-10-30 22:48:15+00:00,2017-11-30 21:59:15+00:00,31


In [36]:
Counter(user_df['total_days'])

Counter({1: 17})

In [51]:
Counter(user_df['total_items'])

Counter({29: 1,
         38: 1,
         56: 1,
         67: 1,
         99: 1,
         100: 1,
         108: 1,
         124: 1,
         130: 1,
         145: 1,
         148: 1,
         158: 1,
         230: 1,
         234: 1,
         280: 1,
         283: 1,
         406: 1,
         462: 1,
         549: 1,
         595: 1,
         596: 1,
         600: 1,
         606: 1,
         632: 1,
         642: 1,
         774: 1,
         810: 2,
         841: 1,
         909: 1,
         1087: 1,
         1114: 1,
         1183: 1,
         1572: 1,
         1660: 1,
         2657: 1,
         14907: 1})

In [5]:
a=[4,5,6]

In [6]:
a.extend(range(5))

In [7]:
a

[4, 5, 6, 0, 1, 2, 3, 4]